In [1]:
import uuid

from datetime import datetime
import values, helpers
import random
import pandas as pd
from create_sap_table import create_table_leanx as ctl

In [2]:
class User:
    def __init__(self, bname, ustyp, mandt=values.mandt) -> None:
        self.mandt = mandt
        self.bname = bname
        self.ustyp = ustyp

class Material:
    def __init__(self, matnr, price, availability, mandt=values.mandt) -> None:
        self.mandt = mandt
        self.matnr = matnr
        self.price = price
        self.availability = availability

class Customer:
    def __init__(self, kunnr, erdat, credit_risk, mandt=values.mandt) -> None:
        self.mandt = mandt
        self.kunnr = kunnr
        self.erdat = erdat
        self.credit_risk = credit_risk
        
class SalesOrderItem:
    def __init__(self, vbeln, posnr, mandt=values.mandt) -> None:
        self.mandt = mandt
        self.vbeln = vbeln
        self.posnr = posnr

In [3]:
cdpos_json = {}
cdhdr_json = {}
vbak_json = {}
vbkd_json = {}
vbuk_json = {}
tvakt_json = {}
tvak_json = {}
tvkot_json = {}
tvtwt_json = {}
tvsbt_json = {}
t001_json = {}
tvkbt_json = {}
dd07t_json = {}
tvlst_json = {}
tvfst_json = {}
vbap_json = {}
tvagt_json = {}
tvapt_json = {}
tvro_json = {}
usr02_json = {}

In [4]:
# insert tables and create objects list
USERS = []
for k, v in values.users.items():
    usr02_json[k] = {
        "MANDT": values.mandt,
        "BNAME": k,
        "USTYP": v
    }

    USERS.append(User(bname=k, ustyp=v))

In [5]:
CUSTOMERS = []
for k, v in values.customers.items():
    rand_kunnr = uuid.uuid4()
    rand_erdat = helpers.generate_random_datetime(start_date=datetime(2022, 1, 1), end_date=datetime(2023, 1, 1))

    CUSTOMERS.append(Customer(kunnr=rand_kunnr, erdat=rand_erdat, credit_risk=v['credit_risk']))

MATERIALS = []
for k, v in values.materials.items():
    mara_last_index = len(MATERIALS)
    rand_matnr = uuid.uuid4()

    MATERIALS.append(Material(matnr=rand_matnr, price=v['price'], availability=v['availability']))

In [6]:
def create_sales_order(vbak_args, vbkd_args, vbuk_args, tvkot_args, tvtwt_args, tvsbt_args, t001_args, tvkbt_args, dd07t_args, tvakt_args):
    vbak_json[vbak_args["VBELN"]] = {
        "MANDT": vbak_args["MANDT"],
        "VBELN": vbak_args["VBELN"],
        "ERDAT": vbak_args["ERDAT"],
        "ERZET": vbak_args["ERZET"],
        "ERNAM": vbak_args["ERNAM"],
        "OBJNR": vbak_args["OBJNR"],
        "AUART": vbak_args['AUART'],
        "BUKRS_VF": vbak_args["BUKRS_VF"] ,
        "KUNNR": vbak_args["KUNNR"],
        "VSBED": vbak_args["VSBED"],
        "VKBUR": vbak_args["VKBUR"],
        "KKBER": vbak_args["KKBER"],
        "NETWR": vbak_args["NETWR"],
        "WAERK": vbak_args["WAERK"],
        "VBTYP": vbak_args["VBTYP"],
        "VDATU": vbak_args["VDATU"],
        "BSTNK": vbak_args["BSTNK"],
        "BSTDK": vbak_args["BSTDK"],
        "VKORG": vbak_args["VKORG"],
        "VTWEG": vbak_args["VTWEG"],
    }
    vbkd_json[f'{vbak_args["VBELN"]}_{"000000"}'] = {
        "MANDT": vbak_args["MANDT"],
        "VBELN": vbak_args["VBELN"],
        "POSNR": '000000',
        "INCO2": vbkd_args["INCO2"],
        "INCO1": vbkd_args["INCO1"],
        "ZTERM": vbkd_args["ZTERM"],
    }
    vbuk_json[vbak_args["VBELN"]] = {
        "MANDT": vbak_args["MANDT"],
        "VBELN": vbak_args["VBELN"],
        "GBSTK": vbuk_args["GBSTK"],
        "BESTK": vbuk_args["BESTK"],
        "CMGST": vbuk_args["CMGST"],
    }
    tvakt_json[f'{"EN"}_{vbak_args["AUART"]}'] = {
        "MANDT": vbak_args["MANDT"],
        "SPRAS": 'EN',
        "AUART": vbak_args["AUART"],
        "BEZEI": tvakt_args["BEZEI"],
    }
    tvak_json[f'{vbak_args["MANDT"]}_{vbak_args["AUART"]}'] = {
        "MANDT": vbak_args["MANDT"],
        "AUART": vbak_args["AUART"],
        "KLIMP": "KLIMP",
    }
    tvkot_json[f'{"EN"}_{vbak_args["VKORG"]}'] = {
        "MANDT": vbak_args["MANDT"],
        "SPRAS": 'EN',
        "VKORG": vbak_args["VKORG"],
        "VTEXT": tvkot_args["VTEXT"],
    }
    tvtwt_json[f'{"EN"}_{vbak_args["VTWEG"]}'] = {
        "MANDT" : vbak_args["MANDT"],
        "SPRAS" : "EN",
        "VTWEG" : vbak_args["VTWEG"],
        "VTEXT" : tvtwt_args["VTEXT"],
    }
    tvsbt_json[f'{"EN"}_{vbak_args["VSBED"]}'] = {
        "MANDT": vbak_args["MANDT"],
        "SPRAS": "EN",
        "VSBED": vbak_args["VSBED"],
        "VTEXT": tvsbt_args["VTEXT"],
    }
    t001_json[f'{vbak_args["BUKRS_VF"]}'] = {
        "MANDT": vbak_args["MANDT"],
        "BUKRS": vbak_args["BUKRS_VF"],
        "BUTXT": t001_args["BUTXT"],
    }
    tvkbt_json[f'{"EN"}_{vbak_args["VKBUR"]}'] = {
        "MANDT": vbak_args["MANDT"],
        "SPRAS": "EN",
        "VKBUR": vbak_args["VKBUR"],
        "BEZEI": tvkbt_args["BEZEI"],
    }
    dd07t_json[f'{"KLIMP"}_{"EN"}'] = {
        "DOMNAME": "VBTYP",
        "DDLANGUAGE": "EN",
        "DOMVALUE_L": vbak_args["VBTYP"],
        "AS4LOCAL": dd07t_args["AS4LOCAL"],
        "VALPOS": dd07t_args["VALPOS"],
        "AS4VERS": dd07t_args["AS4VERS"],
        "DDTEXT": dd07t_args["DDTEXT"],
    }

def create_sales_order_item(vbak_args, vbap_args, tvagt_args, tvapt_args, tvro_args):
    vbap_json[f'{vbak_args["VBELN"]}_{vbap_args["POSNR"]}'] = { 
        "MANDT": vbak_args["MANDT"],
        "VBELN": vbak_args["VBELN"],
        "ERDAT": vbak_args["ERDAT"],
        "ERZET": vbak_args["ERZET"],
        "VGTYP": vbap_args["VGTYP"],
        "VGBEL": vbap_args["VGBEL"],
        "VGPOS": vbap_args["VGPOS"],
        "ERNAM": vbak_args["ERNAM"],
        "NETWR": vbap_args["NETWR"],
        "WAERK": vbap_args["WAERK"],
        "NETPR": vbap_args["NETPR"],
        "KPEIN": vbap_args["KPEIN"],
        "MATNR": vbap_args["MATNR"],
        "WERKS": vbap_args["WERKS"],
        "ABGRU": vbap_args["ABGRU"],
        "OBJNR": vbap_args["OBJNR"],
        "FKREL": vbap_args["FKREL"],
        "PSTYV": vbap_args["PSTYV"],
        "POSNR": vbap_args["POSNR"],
        "BRGEW": vbap_args["BRGEW"],
        "GEWEI": vbap_args["GEWEI"],
        "NTGEW": vbap_args["NTGEW"],
        "VRKME": vbap_args["VRKME"],
        "KWMENG": vbap_args["KWMENG"],
        "KDMAT": vbap_args["KDMAT"],
        "ROUTE": vbap_args["ROUTE"],
    }

    tvagt_json[f'{"EN"}_{vbap_args["ABGRU"]}'] = {
        "MANDT": vbak_args["MANDT"],
        "BEZEI": tvagt_args["BEZEI"],
        "SPRAS": "EN",
        "ABGRU": vbap_args["ABGRU"],
    }
    tvapt_json[f'{"EN"}_{vbap_args["PSTYV"]}'] = {    
        "MANDT": vbak_args["MANDT"],
        "VTEXT": tvapt_args["VTEXT"],
        "SPRAS": "EN",
        "PSTYV": vbap_args["PSTYV"],
    }
    tvro_json[f'{vbap_args["ROUTE"]}'] = {
        "MANDT": vbak_args["MANDT"],
        "TRAZTD": tvro_args["TRAZTD"],
        "ROUTE": vbap_args["ROUTE"]
    }

def set_billing_block(cdhdr_args, cdpos_args, vbak_changed_args):
    # update the coresponding row
    for k, v in vbak_changed_args.items():
        vbak_json[vbak_changed_args['VBELN']][k] = v

    # record change in header
    cdhdr_json[f'{cdhdr_args["MANDANT"]}_{cdhdr_args["OBJECTCLAS"]}_{cdhdr_args["OBJECTID"]}_{cdhdr_args["CHANGENR"]}'] = {
        "MANDANT": cdhdr_args["MANDANT"],
        "UDATE": cdhdr_args["UDATE"],
        "UTIME": cdhdr_args["UTIME"],
        "TCODE": cdhdr_args["TCODE"],
        "OBJECTCLAS": cdhdr_args["OBJECTCLAS"],
        "OBJECTID": cdhdr_args["OBJECTID"],
        "USERNAME": cdhdr_args["USERNAME"],
        "CHANGENR": cdhdr_args["CHANGENR"],
    }

    # record change in postion
    cdpos_json[f'{cdhdr_args["MANDANT"]}_{cdhdr_args["OBJECTCLAS"]}_{cdhdr_args["OBJECTID"]}_{cdhdr_args["CHANGENR"]}_{cdpos_args["TABNAME"]}_{cdpos_args["TABKEY"]}_{cdpos_args["FNAME"]}_{cdpos_args["CHNGIND"]}'] = {
        "MANDANT": cdhdr_args["MANDANT"],
        "OBJECTCLAS": cdhdr_args["OBJECTCLAS"],
        "OBJECTID": cdhdr_args["OBJECTID"],
        "CHANGENR": cdhdr_args["CHANGENR"],
        "TABKEY": cdpos_args["TABKEY"],
        "VALUE_NEW": cdpos_args["VALUE_NEW"],
        "VALUE_OLD": cdpos_args["VALUE_OLD"],
        "CHNGIND": cdpos_args["CHNGIND"],
        "TABNAME": cdpos_args["TABNAME"],
        "FNAME": cdpos_args["FNAME"],
    }

    # update VBAK related tables
    tvfst_json[f'{values.mandt}_{"EN"}_{vbak_changed_args["FAKSK"]}'] = {
        "MANDT": values.mandt,
        "FAKSP": vbak_changed_args["FAKSK"],
        "SPRAS": "EN"
    }

def set_delivery_block(cdhdr_args, cdpos_args, vbak_changed_args):
    # update the coresponding row
    for k, v in vbak_changed_args.items():
        vbak_json[vbak_changed_args['VBELN']][k] = v

    # record change in header
    cdhdr_json[f'{cdhdr_args["MANDANT"]}_{cdhdr_args["OBJECTCLAS"]}_{cdhdr_args["OBJECTID"]}_{cdhdr_args["CHANGENR"]}'] = {
        "MANDANT": cdhdr_args["MANDANT"],
        "UDATE": cdhdr_args["UDATE"],
        "UTIME": cdhdr_args["UTIME"],
        "TCODE": cdhdr_args["TCODE"],
        "OBJECTCLAS": cdhdr_args["OBJECTCLAS"],
        "OBJECTID": cdhdr_args["OBJECTID"],
        "USERNAME": cdhdr_args["USERNAME"],
        "CHANGENR": cdhdr_args["CHANGENR"],
    }

    # record change in postion
    cdpos_json[f'{cdhdr_args["MANDANT"]}_{cdhdr_args["OBJECTCLAS"]}_{cdhdr_args["OBJECTID"]}_{cdhdr_args["CHANGENR"]}_{cdpos_args["TABNAME"]}_{cdpos_args["TABKEY"]}_{cdpos_args["FNAME"]}_{cdpos_args["CHNGIND"]}'] = {
        "MANDANT": cdhdr_args["MANDANT"],
        "OBJECTCLAS": cdhdr_args["OBJECTCLAS"],
        "OBJECTID": cdhdr_args["OBJECTID"],
        "CHANGENR": cdhdr_args["CHANGENR"],
        "TABKEY": cdpos_args["TABKEY"],
        "VALUE_NEW": cdpos_args["VALUE_NEW"],
        "VALUE_OLD": cdpos_args["VALUE_OLD"],
        "CHNGIND": cdpos_args["CHNGIND"],
        "TABNAME": cdpos_args["TABNAME"],
        "FNAME": cdpos_args["FNAME"],
    }

    # update VBAK related tables
    tvlst_json[f'{values.mandt}_{"EN"}_{vbak_changed_args["LIFSK"]}'] = {
        "MANDT": values.mandt,
        "LIFSP": vbak_changed_args["LIFSK"],
        "SPRAS": "EN"
    }

def set_credit_block(vbuk_changed_args, cdhdr_args, cdpos_args):
     # update the coresponding row
    for k, v in vbuk_changed_args.items():
        vbuk_json[vbuk_changed_args['VBELN']][k] = v

    # record change in header
    cdhdr_json[f'{cdhdr_args["MANDANT"]}_{cdhdr_args["OBJECTCLAS"]}_{cdhdr_args["OBJECTID"]}_{cdhdr_args["CHANGENR"]}'] = {
        "MANDANT": cdhdr_args["MANDANT"],
        "UDATE": cdhdr_args["UDATE"],
        "UTIME": cdhdr_args["UTIME"],
        "TCODE": cdhdr_args["TCODE"],
        "OBJECTCLAS": cdhdr_args["OBJECTCLAS"],
        "OBJECTID": cdhdr_args["OBJECTID"],
        "USERNAME": cdhdr_args["USERNAME"],
        "CHANGENR": cdhdr_args["CHANGENR"],
    }

    # record change in postion
    cdpos_json[f'{cdhdr_args["MANDANT"]}_{cdhdr_args["OBJECTCLAS"]}_{cdhdr_args["OBJECTID"]}_{cdhdr_args["CHANGENR"]}_{cdpos_args["TABNAME"]}_{cdpos_args["TABKEY"]}_{cdpos_args["FNAME"]}_{cdpos_args["CHNGIND"]}'] = {
        "MANDANT": cdhdr_args["MANDANT"],
        "OBJECTCLAS": cdhdr_args["OBJECTCLAS"],
        "OBJECTID": cdhdr_args["OBJECTID"],
        "CHANGENR": cdhdr_args["CHANGENR"],
        "TABKEY": cdpos_args["TABKEY"],
        "VALUE_NEW": cdpos_args["VALUE_NEW"],
        "VALUE_OLD": cdpos_args["VALUE_OLD"],
        "CHNGIND": cdpos_args["CHNGIND"],
        "TABNAME": cdpos_args["TABNAME"],
        "FNAME": cdpos_args["FNAME"],
    }

In [7]:
# create sales order
for i in range(100):
    vbak_vbeln = uuid.uuid4()
    customer = random.choice(CUSTOMERS)
    vbak_erdat = helpers.generate_random_datetime(start_date=customer.erdat, end_date=datetime(2024, 1, 1))
    requestd_delivery_time = vbak_erdat + helpers.UPTO_MONTH()
    latest_time = vbak_erdat
    sales_org_key = random.choice(list(values.sales_orgs.keys()))
    sales_office = random.choice(values.sales_orgs[sales_org_key]['sales_offices'])
    
    vbak_args = {
        "MANDT": values.mandt,
        "VBELN": vbak_vbeln,
        "ERDAT": vbak_erdat,
        "ERZET": helpers.generate_random_time(),
        "ERNAM": random.choice(USERS).bname,
        "OBJNR": vbak_vbeln,
        'AUART': random.choice(values.sales_order_doc_types),
        "BUKRS_VF": customer.kunnr, # we are considering each customer to be a legal entity (to simplify)
        "KUNNR": customer.kunnr,
        "VSBED": random.choice(values.shipping_conditions),
        "VKORG": sales_org_key,
        "VKBUR": sales_office,
        "KKBER": random.choice(values.credit_control_areas),
        "NETWR": 0, # FIXME add this value from VBAP
        "WAERK": 'EUR',
        "VBTYP": random.choice(values.sales_order_doc_types),
        "VDATU": requestd_delivery_time,
        "BSTNK": uuid.uuid4(),
        "BSTDK": vbak_erdat,
        "VTWEG": random.choice(values.distribution_channels)
    }

    # SalesOrderItem (VBAP)
    total_netwr = 0
    num_sales_order_item = 10
    materials = random.sample(MATERIALS, num_sales_order_item)
    for m in materials:
        quantity = random.randint(15, 100)
        total_price = m.price * 12 * 12 * quantity
        total_netwr += total_price
        vbap_args = {
            "VGTYP": 'C',
            "VGBEL": 'DEF', #FIXME
            "VGPOS": 99, #FIXME
            "NETWR": total_price,
            "WAERK": 'EUR',
            "NETPR": m.price,
            "KPEIN": 1, # FIXME
            "MATNR": m.matnr,
            "WERKS": 'DEF', #FIXME
            "ABGRU": 'DE', #FIXME
            "OBJNR": 'DEF', #FIXME
            "FKREL": None, #FIXME
            "PSTYV": 'DEF', #FIXME
            "POSNR": MATERIALS.index(m),
            "BRGEW": 99, #FIXME
            "GEWEI": 'KG',
            "NTGEW": 99, #FIXME
            "VRKME": 99, #FIXME
            "ROUTE": 'DEF', #FIXME
            "KWMENG": quantity,
            "KDMAT": m.matnr
        }
        tvagt_args = {
            "BEZEI": 'DEF', #FIXME
        }
        tvapt_args = {
            "VTEXT": 'DEF', #FIXME
        }
        tvro_args = {
            "TRAZTD": 1, #FIXME
        }
        create_sales_order_item(vbak_args=vbak_args, vbap_args=vbap_args, tvagt_args=tvagt_args, tvapt_args=tvapt_args, tvro_args=tvro_args)
    
    # have SO aftyer SO items creation inorder to have total_netwr passed to SO
    # SalesOrder (VBAK)
    vbak_args["NETWR"] = total_netwr
    vbkd_args = {
        "INCO1": random.choice(values.inco1_terms),
        "INCO2": random.choice(values.inco2_terms),
        "ZTERM": random.choice(values.zterms)
    }
    vbuk_args = {
        "GBSTK": None, #FIXME add overall processing status
        "BESTK": None, #FIXME add confirmation status
        "CMGST": None,
    }
    tvakt_args = {
        "BEZEI": 'DEF' #FIXME
    }
    tvkot_args = {
        "VTEXT": 'DEF' #FIXME
    }
    tvtwt_args = {
        "VTEXT" : 'DEF' #FIXME
    }
    tvsbt_args = {
        "VTEXT" : 'DEF' #FIXME
    }
    t001_args = {
        "BUTXT": 'DEF' #FIXME
    }
    tvkbt_args = {
        "BEZEI": 'DEF' #FIXME
    }
    dd07t_args = {
        "AS4LOCAL": 'DEF', #FIXME
        "VALPOS": 99, #FIXME
        "AS4VERS": 99, #FIXME
        "DDTEXT": 'DEF', #FIXME
    }
    create_sales_order(vbak_args=vbak_args, vbkd_args=vbkd_args, vbuk_args=vbuk_args, tvkot_args=tvkot_args, tvtwt_args=tvtwt_args, tvsbt_args=tvsbt_args, t001_args=t001_args, tvkbt_args=tvkbt_args, dd07t_args=dd07t_args, tvakt_args=tvakt_args)
    
    # Set SalesOrde Billing Block
    latest_time += helpers.UPTO_WEEK()
    old_value = None
    new_value = 'SB'
    sbb_vbak_changed_args = {
        "VBELN": vbak_vbeln,
        "FAKSK": new_value
    }
    sbb_cdhdr_args = {
        "MANDANT": values.mandt,
        "UDATE": latest_time,
        "UTIME": helpers.generate_random_time(),
        "TCODE": 'CHNGSO',
        "OBJECTCLAS": 'VBAK',
        "OBJECTID": vbak_vbeln,
        "USERNAME": random.choice(USERS).bname,
        "CHANGENR": uuid.uuid4()
    }
    sbb_cdpos_args = {
        "TABKEY": vbak_vbeln,
        "VALUE_NEW": new_value,
        "VALUE_OLD":old_value,
        "CHNGIND": 'U',
        "TABNAME": 'VBAK',
        "FNAME": 'FAKSK'
    }
    set_billing_block(cdhdr_args=sbb_cdhdr_args, cdpos_args=sbb_cdpos_args, vbak_changed_args=sbb_vbak_changed_args)

    # Set SalesOrde Delivery Block
    latest_time += helpers.UPTO_WEEK()
    old_value = None
    new_value = 'SB'
    sdb_vbak_changed_args = {
        "VBELN": vbak_vbeln,
        "LIFSK": new_value
    }
    sdb_cdhdr_args = {
        "MANDANT": values.mandt,
        "UDATE": latest_time,
        "UTIME": helpers.generate_random_time(),
        "TCODE": 'CHNGSO',
        "OBJECTCLAS": 'VBAK',
        "OBJECTID": vbak_vbeln,
        "USERNAME": random.choice(USERS).bname,
        "CHANGENR": uuid.uuid4()
    }
    sdb_cdpos_args = {
        "TABKEY": vbak_vbeln,
        "VALUE_NEW": new_value,
        "VALUE_OLD":old_value,
        "CHNGIND": 'U',
        "TABNAME": 'VBAK',
        "FNAME": 'LIFSK'
    }
    set_delivery_block(cdhdr_args=sdb_cdhdr_args, cdpos_args=sdb_cdpos_args, vbak_changed_args=sdb_vbak_changed_args)

    # Set SalesOrde Credit Block
    latest_time += helpers.UPTO_WEEK()
    old_value = None
    new_value = 'B'
    scb_vbuk_changed_args = {
        "VBELN": vbak_vbeln,
        "CMGST": new_value
    }
    scb_cdhdr_args = {
        "MANDANT": values.mandt,
        "UDATE": latest_time,
        "UTIME": helpers.generate_random_time(),
        "TCODE": 'CHNGHS',
        "OBJECTCLAS": 'VBUK',
        "OBJECTID": vbak_vbeln,
        "USERNAME": random.choice(USERS).bname,
        "CHANGENR": uuid.uuid4()
    }
    scb_cdpos_args = {
        "TABKEY": vbak_vbeln,
        "VALUE_NEW": new_value,
        "VALUE_OLD":old_value,
        "CHNGIND": 'U',
        "TABNAME": 'VBUK',
        "FNAME": 'CMGST'
    }
    set_credit_block(cdhdr_args=scb_cdhdr_args, cdpos_args=scb_cdpos_args, vbuk_changed_args=scb_vbuk_changed_args)

In [8]:
usr02_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('usr02')])
usr02 = pd.concat([usr02_full_col, pd.DataFrame(usr02_json.values())])

vbak_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('vbak')])
vbak = pd.concat([vbak_full_col, pd.DataFrame(vbak_json.values())])
vbkd_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('vbkd')])
vbkd = pd.concat([vbkd_full_col, pd.DataFrame(vbkd_json.values())])
vbuk_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('vbuk')])
vbuk = pd.concat([vbuk_full_col, pd.DataFrame(vbuk_json.values())])
tvakt_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('tvakt')])
tvakt = pd.concat([tvakt_full_col, pd.DataFrame(tvakt_json.values())])
tvak_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('tvak')])
tvak = pd.concat([tvak_full_col, pd.DataFrame(tvak_json.values())])
tvkot_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('tvkot')])
tvkot = pd.concat([tvkot_full_col, pd.DataFrame(tvkot_json.values())])
tvtwt_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('tvtwt')])
tvtwt = pd.concat([tvtwt_full_col, pd.DataFrame(tvtwt_json.values())])
tvsbt_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('tvsbt')])
tvsbt = pd.concat([tvsbt_full_col, pd.DataFrame(tvsbt_json.values())])
t001_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('t001')])
t001 = pd.concat([t001_full_col, pd.DataFrame(t001_json.values())])
tvkbt_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('tvkbt')])
tvkbt = pd.concat([tvkbt_full_col, pd.DataFrame(tvkbt_json.values())])
dd07t_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('dd07t')])
dd07t = pd.concat([dd07t_full_col, pd.DataFrame(dd07t_json.values())])
tvlst_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('tvlst')])
tvlst = pd.concat([tvlst_full_col, pd.DataFrame(tvlst_json.values())])
tvfst_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('tvfst')])
tvfst = pd.concat([tvfst_full_col, pd.DataFrame(tvfst_json.values())])

vbap_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('vbap')])
vbap = pd.concat([vbap_full_col, pd.DataFrame(vbap_json.values())])
tvagt_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('tvagt')])
tvagt = pd.concat([tvagt_full_col, pd.DataFrame(tvagt_json.values())])
tvapt_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('tvapt')])
tvapt = pd.concat([tvapt_full_col, pd.DataFrame(tvapt_json.values())])
tvro_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('tvro')])
tvro = pd.concat([tvro_full_col, pd.DataFrame(tvro_json.values())])

cdpos_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('cdpos')])
cdpos = pd.concat([cdpos_full_col, pd.DataFrame(cdpos_json.values())])
cdhdr_full_col = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table('cdhdr')])
cdhdr = pd.concat([cdhdr_full_col, pd.DataFrame(cdhdr_json.values())])

In [9]:
cdpos.to_csv('data/ocpm/cdpos.csv', index=False)
cdhdr.to_csv('data/ocpm/cdhdr.csv', index=False)
vbak.to_csv('data/ocpm/vbak.csv', index=False)
vbkd.to_csv('data/ocpm/vbkd.csv', index=False)
vbuk.to_csv('data/ocpm/vbuk.csv', index=False)
tvakt.to_csv('data/ocpm/tvakt.csv', index=False)
tvak.to_csv('data/ocpm/tvak.csv', index=False)
tvkot.to_csv('data/ocpm/tvkot.csv', index=False)
tvtwt.to_csv('data/ocpm/tvtwt.csv', index=False)
tvsbt.to_csv('data/ocpm/tvsbt.csv', index=False)
t001.to_csv('data/ocpm/t001.csv', index=False)
tvkbt.to_csv('data/ocpm/tvkbt.csv', index=False)
dd07t.to_csv('data/ocpm/dd07t.csv', index=False)
vbap.to_csv('data/ocpm/vbap.csv', index=False)
tvagt.to_csv('data/ocpm/tvagt.csv', index=False)
tvapt.to_csv('data/ocpm/tvapt.csv', index=False)
tvro.to_csv('data/ocpm/tvro.csv', index=False)
usr02.to_csv('data/ocpm/usr02.csv', index=False)
tvlst.to_csv('data/ocpm/tvlst.csv', index=False)
tvfst.to_csv('data/ocpm/tvfst.csv', index=False)

In [10]:
cdpos

,MANDANT,OBJECTCLAS,OBJECTID,CHANGENR,TABNAME,TABKEY,FNAME,CHNGIND,TEXT_CASE,UNIT_OLD,UNIT_NEW,CUKY_OLD,CUKY_NEW,VALUE_NEW,VALUE_OLD
0,SC1,VBAK,58750934-8fc9-4097-9bf4-83f0e9818244,96cb7e1e-b453-4a8d-b8c9-fcb586607740,VBAK,58750934-8fc9-4097-9bf4-83f0e9818244,FAKSK,U,NaN,NaN,NaN,NaN,NaN,SB,None
1,SC1,VBAK,58750934-8fc9-4097-9bf4-83f0e9818244,066c385b-3330-48ec-878e-c4d4cb134a58,VBAK,58750934-8fc9-4097-9bf4-83f0e9818244,LIFSK,U,NaN,NaN,NaN,NaN,NaN,SB,None
2,SC1,VBUK,58750934-8fc9-4097-9bf4-83f0e9818244,e6bc29d8-496d-43fe-8f2c-6ece416d5685,VBUK,58750934-8fc9-4097-9bf4-83f0e9818244,CMGST,U,NaN,NaN,NaN,NaN,NaN,B,None
3,SC1,VBAK,87c02070-3b7b-4665-94d3-5a753097ab74,bf90869c-70c8-4c9c-8c76-4e6842e6c8ab,VBAK,87c02070-3b7b-4665-94d3-5a753097ab74,FAKSK,U,NaN,NaN,NaN,NaN,NaN,SB,None
4,SC1,VBAK,87c02070-3b7b-4665-94d3-5a753097ab74,e2a6002d-571b-455c-baf6-6c3131b23e65,VBAK,87c02070-3b7b-4665-94d3-5a753097ab74,LIFSK,U,NaN,NaN,NaN,NaN,NaN,SB,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,SC1,VBAK,052bd89c-8b62-43b6-b417-82ce8b3d1369,db181593-53f3-48a2-aa1a-29467db35bd4,VBAK,052bd89c-8b62-43b6-b417-82ce8b3d1369,LIFSK,U,NaN,NaN,NaN,NaN,NaN,SB,None
296,SC1,VBUK,052bd89c-8b62-43b6-b417-82ce8b3d1369,79738ed3-e5f0-4ff1-b5b8-5dac26b305a7,VBUK,052bd89c-8b62-43b6-b417-82ce8b3d1369,CMGST,U,NaN,NaN,NaN,NaN,NaN,B,None
297,SC1,VBAK,454b4b11-f223-4e3a-8224-1cc4783f0575,f22c0df5-f6d1-42bb-8c1d-402e3a111115,VBAK,454b4b11-f223-4e3a-8224-1cc4783f0575,FAKSK,U,NaN,NaN,NaN,NaN,NaN,SB,None
298,SC1,VBAK,454b4b11-f223-4e3a-8224-1cc4783f0575,f0a3a47f-279c-45f6-85cf-3c1376c508b1,VBAK,454b4b11-f223-4e3a-8224-1cc4783f0575,LIFSK,U,NaN,NaN,NaN,NaN,NaN,SB,None
